# Tools LangChain Agent

In this notebook, we explore how to set up a LangChain agent that can use external tools to enhance its capabilities. This type of agent is called a "Tool Calling Agent." It can perform tasks such as fetching data from the web, querying APIs, or accessing specialized knowledge bases. 

## Overview

1. **What is a Tool Calling Agent?**
2. **Setting Up the Environment**
3. **Creating and Configuring the Agent**
4. **Using External Tools with the Agent**
5. **Examples of Tool Usage**

## 1. What is a Tool Calling Agent?

A **Tool Calling Agent** in LangChain is an AI agent that can call external tools to perform specific tasks. These tools can include APIs, databases, or other services that provide additional data or functionality. By integrating these tools, the agent can go beyond its built-in capabilities and provide more accurate or specialized responses.

## 2. Setting Up the Environment

Before we create our agent, we need to install some necessary libraries. These libraries will allow us to create the agent, connect to external tools, and manage data.

### Install Necessary Libraries

To get started, install the following libraries using pip:

In [ ]:
# Install necessary libraries
%pip install langchain langchain_community langchain_core langchainhub langchain-ollama faiss-cpu arxiv wikipedia

## 3. Creating and Configuring the Agent

In this section, we will create the main components of our agent, including the language model and any required tools

### Import Necessary Modules

First, we import the necessary modules from the `langchain`, `langchain_ollama`, and `langchain_community` packages. These modules include tools for loading models, fetching data, and handling various tasks.

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import  ArxivQueryRun
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain import hub

### Setting Up the Language Model

We set up the language model (LLM) that the agent will use to process natural language. Here, we use the `ChatOllama` model, a versatile LLM capable of handling various text-based tasks.

In [300]:
llm = ChatOllama(
    model="llama3-groq-tool-use",
    # Optional parameters to fine-tune model behavior
    temperature=1,  # Controls randomness in responses
    max_new_tokens=512  # Limits the length of the output
)

### Setting Up External Tools

External tools can be anything from APIs to databases that provide data or perform specific tasks. Here, we set up some tools like `WikipediaAPIWrapper` and `ArxivAPIWrapper` for fetching information from Wikipedia and ArXiv, respectively.

#### Document Loader and Retriever Tool Setup

We use the `WebBasedLoader` to load documents from a specified URL and `FAISS` for vector-based similarity search.

In [291]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(
    documents=docs[:30],
     embedding=OllamaEmbeddings(model="dolphin-llama3",show_progress=True),
)
retriever = vectordb.as_retriever()
retriever

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x105068850>)

In [292]:
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

retriever_tool.name

'langsmith_search'

#### Wikipedia Tool Setup & ArXiv Tool Setup

In [293]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [294]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

#### Define the List of Tools

Now that we have defined our tools, we can combine them into our agent. The agent will use these tools to handle specific types of queries, enhancing its capabilities to provide more accurate and relevant information.

We define a list of the tools we created, which the agent can call upon as needed.

In [295]:
tools = [wiki, arxiv, retriever_tool]

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/gsampaio/redhat/ai/llm-agents/.conda/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10f6922a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.

#### Getting the Prompt Template

The prompt template defines the structure of the questions or instructions given to the agent. We can use an existing prompt template from a library or customize our own.

In [296]:
from langchain.tools.render import render_text_description_and_args
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# chat_model_with_stop = llm.bind(stop=["\nObservation"])

# Inspiration taken from hub.pull("hwchase17/react-json")
system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: 
Observation: the result of the action... 
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

print(system_message)

# Here, we are using prompt template that is suitable for working with the tools we have integrated. You can modify this prompt template as needed to better fit your specific use case.

Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {{'query': {{'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}}}
arxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query., args: {{'query': {{'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}}}
langsmith_search(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<la

## 4. Creating and Using the Tool Calling Agent

### Creating the Agent

We create the Tool Calling Agent using the `create_tool_calling_agent` function, which integrates the language model, tools, and prompt template.

In this setup:

- **llm**: The language model used to interpret and generate text.
- **tools**: The list of tools that the agent can use.
- **prompt**: The prompt template defining the agent's input format.

In [301]:
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")

# prompt.messages
# prompt.pretty_print()

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Using the Agent

Finally, we can use the agent to process a specific input query. The agent will decide which tools to use based on the query and provide a response.

In [309]:
# In this example, the agent is queried with "Tell me about Langsmith and then search on wikipedia for Nautico Capibaribe. It will use the appropriate tools to gather information and provide an answer.
agent_executor.invoke({"input":"Tell me about Langsmith and then search on wikipedia for Nautico Capibaribe. At the end write the two answers!"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.84it/s]







Get started with LangSmith | 🦜️🛠️ LangSmith







Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide multiple ways to log traces to LangSmith. Below, we'll highlight
how to use 

{'input': 'Tell me about Langsmith and then search on wikipedia for Nautico Capibaribe. At the end write the two answers!',
 'output': 'Here are the answers:\nLangsmith: LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nNautico Capibaribe: Clube Náutico Capibaribe (Portuguese pronunciation: [ˈklubi ˈnawtʃiku kɐpibɐˈɾibi]), or simply Náutico, is a Brazilian multi-sport club based in Recife, Pernambuco.'}

## Defining Custom Tools

In [310]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def calculate_length_tool(a: str) -> int:
    """The function calculates the length of the input string."""
    return len(a)

calculate_length_tool.description, calculate_length_tool.args,

('The function calculates the length of the input string.',
 {'a': {'title': 'A', 'type': 'string'}})

In [311]:
# Add the calculate_length_tool to the list
tools = [wiki, arxiv, retriever_tool, calculate_length_tool]

system_message = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

{render_text_description_and_args(tools).replace('{', '{{').replace('}', '}}')}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use)
and a `action_input` key (with the input to the tool going here).
The only values that should be in the "action" field are: {[t.name for t in tools]}
The $JSON_BLOB should only contain a SINGLE action, 
do NOT return a list of multiple actions.
Here is an example of a valid $JSON_BLOB:
```
{{{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}}}
```
The $JSON_BLOB must always be enclosed with triple backticks!

ALWAYS use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action:```
$JSON_BLOB
```
Observation: the result of the action... 
(this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            system_message,
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

print(system_message)

# tools

Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherise, you have access to the following tools:

wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query., args: {{'query': {{'title': 'Query', 'description': 'query to look up on wikipedia', 'type': 'string'}}}}
arxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query., args: {{'query': {{'title': 'Query', 'description': 'search query to look up', 'type': 'string'}}}}
langsmith_search(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<la

In [316]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input":"Tell me about Langsmith and calculate the length of the text result."})



> Entering new AgentExecutor chain...
Question: What can you tell me about Langsmith?

> Finished chain.


{'input': 'Tell me about Langsmith and calculate the length of the text result.',
 'output': 'Question: What can you tell me about Langsmith?'}

In [ ]:

# # Using with chat history
# from langchain_core.messages import AIMessage, HumanMessage
# agent_executor.invoke(
#     {
#         "input": "what's my name?",
#         "chat_history": [
#             HumanMessage(content="hi! my name is bob"),
#             AIMessage(content="Hello Bob! How can I assist you today?"),
#         ],
#     }
# )